In [46]:
import datasets as ds
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
import itertools
import os
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
CUR_DIR = os.path.abspath(os.curdir)
ROOT_DIR = os.path.dirname(CUR_DIR)
IMAGES_DIR = os.path.join(ROOT_DIR, "images")
DATA_DIR = os.path.join(ROOT_DIR, "data")
DATA_DIR

'/home/jovyan/projects/vector-nlp/data'

In [3]:
#ds.list_datasets()

In [49]:
app_dataset = ds.load_dataset('app_reviews',split='train')
app_df = app_dataset.to_pandas()
app_df['review_ls'] = app_df['review'].apply(lambda x: x.split(" "))
vocabulary = list(itertools.chain(*app_df['review_ls']))

Using custom data configuration default
Reusing dataset app_reviews (/home/jovyan/.cache/huggingface/datasets/app_reviews/default/0.0.0/20335b51b604b9bc04b7be253cd8445caa9ba93f15f39a4b0492b9e9102853de)


In [29]:
app_df['review_clean'] = app_df['review'].str.replace('\W', ' ')

/tmp/ipykernel_53/2924524219.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  app_df['review_clean'] = app_df['review'].str.replace('\W', ' ')


In [30]:
lemmatizer = WordNetLemmatizer()
def lemma_sentences(sentence):
    tokens = sentence.split()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

app_df['review_lemma'] = app_df['review_clean'].apply(lemma_sentences)

In [31]:
porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

app_df['review_stem'] = app_df['review_clean'].apply(stem_sentences)
app_df['review_stem']

0         great app the new version now work on my bravi...
1         great it s not fulli optimis and ha some issu ...
2         work on a nexu 6p i m still mess around with m...
3         the bandwidth seem to be limit to maximum 2 mh...
4         work well with my hackrf hope new updat will a...
                                ...                        
288060    it doesn t do anyth after instal thi it didn t...
288061               i like thi app it is veri help for use
288062    final bring back the unix command line to android
288063    the api featur is great just need load more fe...
288064    work nice i wish there were instruct that woul...
Name: review_stem, Length: 288065, dtype: object

In [36]:
# X = app_df['review_lemma']
X = app_df['review_stem']
y = app_df['star']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)
X_val = vect.transform(X_val)

In [37]:
X_train

<172839x16767 sparse matrix of type '<class 'numpy.float64'>'
	with 1008770 stored elements in Compressed Sparse Row format>

In [39]:
from collections import Counter
results = Counter(app_df.review_clean)
VOCAB_SIZE = len(results)
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)

2022-03-06 23:50:43.794297: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-06 23:50:43.795158: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-06 23:50:43.795995: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (42b10f96906e): /proc/driver/nvidia/version does not exist
2022-03-06 23:50:43.809637: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [42]:
from collections import Counter
c = Counter()
app_df.review_clean.str.split().apply(c.update)

0         None
1         None
2         None
3         None
4         None
          ... 
288060    None
288061    None
288062    None
288063    None
288064    None
Name: review_clean, Length: 288065, dtype: object

In [40]:
results

Counter({'Great app  The new version now works on my Bravia Android TV which is great as it s right by my rooftop aerial cable  The scan feature would be useful   any ETA on when this will be available  Also the option to import a list of bookmarks e g  from a simple properties file would be useful ': 1,
         'Great It s not fully optimised and has some issues with crashing but still a nice app  especially considering the price and it s open source ': 1,
         'Works on a Nexus 6p I m still messing around with my hackrf but it works with my Nexus 6p  Trond usb c to usb host adapter  Thanks ': 1,
         'The bandwidth seemed to be limited to maximum 2 MHz or so  I tried to increase the bandwidth but not possible  I purchased this is because one of the pictures in the advertisement showed the 2 4GHz band with around 10MHz or more bandwidth  Is it not possible to increase the bandwidth  If not  it is just the same performance as other free APPs ': 1,
         'Works well with my 

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)

In [18]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [19]:
import random

for i in range(10):
    random_id = random.randint(0,len(vect.get_feature_names()))
    print(vect.get_feature_names()[random_id])

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


easi perfect
emulator howev
tabloid
releas just
ha essenti
look online
closing
plz upload
doe control
ois


In [20]:
first_topic = LDA.components_[0]

In [21]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['nice', 'app', 'veri nice', 'easi', 'amaz', 'veri', 'work', 'ok', 'great', 'love']


Top 10 words for topic #1:
['phone', 'hi', 'use', 'wow', 'work', 'best app', 'thi', 'updat', 'app', 'best']


Top 10 words for topic #2:
['happi', 'awesom app', 'slow', 'good good', 'app', 'veri', 'good app', 'veri good', 'awesom', 'good']


Top 10 words for topic #3:
['superb', 'work', 'bad', 'perfect', 'game', 'app', 'nice app', 'use', 'super', 'nice']


Top 10 words for topic #4:
['googl', 'great app', 'use', 'great', 'excel', 'cool', 'thi', 'thank', 'app', 'like']




In [22]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(288065, 5)

In [23]:
app_df['Topic'] = topic_values.argmax(axis=1)

In [24]:
app_df.to_csv(os.path.join(DATA_DIR, "1.1_LDA-Tfidf.csv"),index = False)